In [45]:
# Laster inn nødvendige bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Husk å legge dette inn i txt dokument

In [31]:
# Henter inn data
data = pd.read_csv('../data/compas-scores-two-years.csv')
# Fjerner de radene som det var oppgitt i oppgaven at kunne være dårlige
data = data[data["days_b_screening_arrest"] <= 30]
data = data[data["days_b_screening_arrest"] >= -30]

In [32]:
# Vi tar en titt på hvordan dataen ser ut
data.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
5,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,...,1,Low,2013-11-30,2013-11-30,2013-12-01,0,1,853,0,0
6,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,...,2,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1


In [33]:
# Ser på hvilke kolonner vi har og størrelsen på datasettet.
print(f'Datasettet har størrelse (rader, kolonner): {data.shape}.')
print(f'Kolonnene i datasettet er: {data.columns}')

Datasettet har størrelse (rader, kolonner): (6172, 53).
Kolonnene i datasettet er: Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_ye

#### Dette er hvilke kolloner vi ønsker å fjerne og hvorfor:

* id, name, first, last: Navn og et tilfeldig tildelt nummer har ingenting med analysen å gjøre. 

* compas_screening_date, screening_date, v_screening_date: Det kan jo ha vært endringer i algoritmen over tid, men det er egt ikke det vi setter fokus på. 

* dob, age_cat: Den spesifikke fødselsdatoen er mindre relevant enn forsvarerens alder og alderskategori er mindre nøyaktig. 

* c_case_number, r_case_number, vr_case_number: Spesifikke saksnummer bidrar ikke til å se på trender i dataen. 

* c_jail_in, c_jail_out, r_jail_in, r_jail_out, in_custody, out_custody: Spesifikke fengselsoppholdsdatoer er ikke viktige i analysen. 

* c_offense_date, c_arrest_date, r_offense_date, vr_offense_date: De spesifikke datoene for lovbrudd og pågripelser er heller ikke særlig relevante. 

* c_days_from_compas, r_days_from_arrest, days_b_screening_arrest: Disse kolonnene er ikke være direkte relevante, med mindre analysen spesifikt ser på tidsrommet mellom vurderinger, pågripelser og lovbrudd.

* c_charge_desc, r_charge_desc, vr_charge_desc: Individuelle beskrivelser av anklager kan være for detaljerte for brede analyser.

* type_of_assessment, v_type_of_assessment: Dette er det samme for alle 'Risk of Recidivism' og 'Risk of Violence'.

* priors_count.1, decile_score.1: Ser ut til å være kopier av priors_count og decile_score, så er unødvendige.

* violent_recid: Den er NaN i alle rader

In [34]:
remove = ['id', 'name', 'first', 'last', 'compas_screening_date', 'screening_date', 'v_screening_date', 
    'dob', 'age_cat', 'c_case_number', 'r_case_number', 'vr_case_number', 'c_jail_in', 'c_jail_out', 
    'r_jail_in', 'r_jail_out', 'in_custody', 'out_custody', 'c_offense_date', 'c_arrest_date', 
    'r_offense_date', 'vr_offense_date', 'c_days_from_compas', 'days_b_screening_arrest', 'r_days_from_arrest', 
    'c_charge_desc', 'r_charge_desc', 'vr_charge_desc', 'priors_count.1', 'decile_score.1', 'type_of_assessment', 'v_type_of_assessment', 'violent_recid']

data.drop(columns=remove, inplace=True)


#### Kolonene som vi da har igjen er:

* sex: Kjønnet til individet (male eller female).

* age: Alderen til individet.

* race: Etnisiteten til individet.

* juv_fel_count: Antall ganger individet har blitt dømt for forbrytelser som ungdom (juvenile felony).

* decile_score: En risikoscore gitt av COMPAS, på en skala fra 1 til 10, der 10 indikerer høyest risiko for generell recidivism (begå nytt lovbrudd).

* juv_misd_count: Antall ganger individet har blitt dømt for mindre alvorlige forbrytelser (misdemeanors) som ungdom.

* juv_other_count: Antall andre ganger individet har hatt andre juridiske problemer som ungdom.

* priors_count: Totalt antall tidligere domfellelser.

* c_charge_degree: Graden av alvorlighet for anklagelsen, F for felony og M for misdemeanor. 

* is_recid: Har det vært nytt lovbrudd. 0 = Nei, 1 = Ja.

* r_charge_degree: Graden av anklagen for tilbakefall, med lignende kategorier som c_charge_degree.

* is_violent_recid: En binær indikator som viser om individet har begått en voldelig forbrytelse som recidivism. 1 for ja, 0 for nei.

* vr_charge_degree: Graden av den voldelige nye anklagelsen, lignende kotegori som c_charge_degree og r_charge_degree.

* score_text: Tekst for hvor sannsynlig nytt lovbrudd er, kategorisert som Low, Medium, eller High risiko.

* v_decile_score: En risikoscore gitt av COMPAS for potensiell voldelig nytt lovbrudd, på en skala fra 1 til 10.

* v_score_text: Tekst for hvor sannsynlig nytt voldelig lovbrudd er, kategorisert som Low, Medium, eller High risiko.

* start: Starttidspunktet for den perioden som blir studert.

* end: Sluttpunktet for observasjonsperioden, enten datoen for tilbakefall eller slutten av oppfølgingsperioden.

* event: En binær indikator for om tilbakefall har skjedd i løpet av studieperioden. 1 for ja, 0 for nei.

* two_year_recid: En spesifikk binær indikator som viser om individet har gjentatt kriminell aktivitet innen to år etter risikovurderingen. 1 for ja, 0 for nei. (er det det samem som den is_recid og event?)


Det er ikke sikkert at alle disse kollonenne kommer til å bli brukt, men de er med for nå fordi det er mulig at de kan inneholde nyttig informasjon

*Fjern de som ikke brukes før vi blir ferdige*

In [35]:
# data[data['vr_charge_degree'].notna()]
data.head()

,sex,age,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,c_charge_degree,is_recid,r_charge_degree,is_violent_recid,vr_charge_degree,score_text,v_decile_score,v_score_text,start,end,event,two_year_recid
0,Male,69,Other,0,1,0,0,0,F,0,NaN,0,NaN,Low,1,Low,0,327,0,0
1,Male,34,African-American,0,3,0,0,0,F,1,(F3),1,(F3),Low,1,Low,9,159,1,1
2,Male,24,African-American,0,4,0,1,4,F,1,(M1),0,NaN,Low,3,Low,0,63,0,1
5,Male,44,Other,0,1,0,0,0,M,0,NaN,0,NaN,Low,1,Low,1,853,0,0
6,Male,41,Caucasian,0,6,0,0,14,F,1,(F2),0,NaN,Medium,2,Low,5,40,1,1


# Predikert og faktisk gjentatt forbrytelse

Vi ønsker nå å sammenligne hvor stor andel av ulike grupper som faktisk gjentar forbrytelser og hvor mye det er predikert at de vil det. Da ser vi på gruperingene kvinner, menn, hvite og svarte. 

Vi klassifiserer det slik at de individene som fikk middels eller høy score for sannsynligheten for å gjenta en forbrytelse blir kategorisert som at algoritmen predikerer at det vil begås et nytt lovbrudd, mens de som fikk lav score ikke vil det. Vi sammenligner dette med hva de ulike individene faktisk gjore, da ser vi på om de begikk et nytt lovbrudd i løpet av de to påfølgende årene etter at de ble analysert av algoritmen.

In [58]:
# Legger inn slik at de med middels og høy score får positis på prediksjonen av nytt lovbrudd
data['pred_recid'] = np.where(data['score_text'].isin(['High', 'Medium']), 1, 0)

# Predikert og faktisk gjentagelse av lovbrudd basert på kjønn og rase

In [60]:
# Regner ut predikert og faktisk verdi for gjentagelse for de ulike gruppene
andel_grupert = data.groupby(['sex', 'race']).agg({
    'pred_recid': lambda x: (x == 1).mean() * 100,
    'two_year_recid': lambda x: (x == 1).mean() * 100
}).reset_index()

# Tar kun med hvite og sorte individer
andel_grupert = andel_grupert[andel_grupert['race'].isin(['Caucasian','African-American'])]

# Regner ut differansen mellom predikert og faktisk
andel_grupert['forskjell'] = andel_grupert['pred_recid'] - andel_grupert['two_year_recid']

display(andel_grupert)

,sex,race,pred_recid,two_year_recid,forskjell
0,Female,African-American,49.544627,36.976321,12.568306
2,Female,Caucasian,38.174274,35.269710,2.904564
6,Male,African-American,59.291698,55.521706,3.769992
8,Male,Caucasian,31.585441,40.222085,-8.636644


Vi kan se i tabellen over at det er størt forskjell blandt kvinnlige afro-amerikanere, der det er predikert at hele 12.6 prosentpoeng flere ville begå en ny straffar handling ann de som faktisk gjorge det. For hvite kvinner og sorte menn ble der predikert henholdsvis 2.9 og 3.8 prosentpoeng flere enn det det faktisk var. Men når det kommer til hvite menn ble det motsatt. Her ble det predikert at 8.6 prosentpoeng færre skulle begå et nytt lovbrudd enn det som faktisk skjedde. 

Den gruppen som begikk flest nye lovbrudd var sorte menn, som var det samme som det som ble predikert. Derimot ble det predikert at den gruppen som ville begå færrest nye lovbrudd var hvite menn, men det var faktisk hvite kvinner, men sorte kvinner rett bak, som faktisk begikk begikk færrest nye lovbrudd. 

# Risikoscore basert på kjønn og rase

Her ser vi på hvilken risikoscore de ulike gruppene fikk basert på deres kjønn og rase.

In [85]:
# Tar kun med hvite og sorte individer
risikoscore = data[data['race'].isin(['Caucasian','African-American'])]

# Legger sammen hvor mange i hver gruppe fikk de ulike scorene
andel_score = risikoscore.groupby(['sex', 'race'])['score_text'].value_counts(normalize=True).unstack(fill_value=0) * 100

# Viser tabell
andel_score


score_text                    High        Low     Medium
sex    race                                             
Female African-American  17.850638  50.455373  31.693989
       Caucasian         10.373444  61.825726  27.800830
Male   African-American  28.446306  40.708302  30.845392
       Caucasian         10.672424  68.414559  20.913017

I denne tabellen kan vi se at hvite kvinner er den gruppen som mest skjeldent får scoren høy (hvite menn er bare litt mer) og det er sorte menn som oftest får denne scoren. 

Det er sorte kvinner som oftest får scoren medium (rett foran sorte menn), mens det er hvite menn som mest skjeldent får denne scoren.

For lav score, er det hvite kvinner som oftest får denne scoren. Det er sorte menn som mest skjelden får lav score.

# Risikoscore basert på om det ble begått ny straffbar handling

Her ser vi på om det har blitt begått en ny straffbar handling eller ikke, og hvilken risikoscore som ble tildelt.

In [83]:
andel_recid_score = data.groupby(['two_year_recid'])['score_text'].value_counts(normalize=True).unstack(fill_value=0) * 100

# Viser tabell
andel_recid_score

score_text,High,Low,Medium
two_year_recid,,,
0,8.831401,69.729408,21.439191
1,30.153079,38.305447,31.541474


Her kan vi se at iblant de som ikke begykk en ny straffber handling hadde 8.8% go 21% motatt høy og medium score, mens hele 70% mottok lav score. Vi kan sammenligne dette med de som faktisk begikk en ny straffbar handling. i den gruppen var det 30% og 32% som fikk høy og medium score, men bare 38% mottok lav score. 

Dermed kan vi se at det er en signifikant forskjell mellom de to gruppene. Vi kan se at de som begikk en ny straffbar handling generelt hadde fått en høyere score enn de som ikke gjorde det. Det er også enkelt å se at det var en god del som ble feil kategorisert, ved at de fikk lav score og begikk en ny straffbar handling eller motsatt. 

Nå skal vi undersøke hva som skjer dersom vi også tar en titt på hvilken etnisitet disse individene hadde.

In [84]:
# Tar kun med hvite og sorte individer
recid_race_score = data[data['race'].isin(['Caucasian','African-American'])]

# Grupperer etter 'two_year_recid' og 'race', olg legger sammen antall av hver score i gruperingen
recid_race_score = recid_race_score.groupby(['two_year_recid', 'race'])['score_text'].value_counts(normalize=True).unstack(fill_value=0) * 100

# Viser tabell
recid_race_score

score_text                            High        Low     Medium
two_year_recid race                                             
0              African-American  13.936592  57.661823  28.401585
               Caucasian          4.761905  77.985948  17.252147
1              African-American  38.169777  28.476821  33.353402
               Caucasian         19.708029  49.635036  30.656934

Når vi nå har tatt hensyn til rase kan vi se at det er ganske klare forskjeller. Selv om vi kan generellt se de samme trendene i score som når vi ikke så på rase, kan vi se at for både de som begykk ny kriminatitet og de som ikke, er scoren for de sorte har blitt forskjøvet mot høyere score, mens den har blitt forskjøvet andre veien for de hvite. Dette kan indikere forskjellsbehandling mellom rasene og kan vise til at de sorte oppfører seg bedre enn forventet og hvite verre enn forventet generellt. 

# Rater av resultater besert på grupper

Her ser vi på hvor mange falske positive, falske negative, sanne positive og sanne negative, samt positiv prediktiv verdi vi har for de ulike gruppene: hele befolkningen, kvinner, menn, hvite og svarte.

In [132]:
def rate(subset):
    TP = len(subset[(subset['two_year_recid'] == 1) & (subset['pred_recid'] == 1)])
    FP = len(subset[(subset['two_year_recid'] == 0) & (subset['pred_recid'] == 1)])
    TN = len(subset[(subset['two_year_recid'] == 0) & (subset['pred_recid'] == 0)])
    FN = len(subset[(subset['two_year_recid'] == 1) & (subset['pred_recid'] == 0)])
    
    TPR = TP/(TP+FN)*100
    FPR = FP/(FP+TN)*100
    TNR =TN/(TN+FP)*100
    FNR = FN/(FN+TP)*100

    pos_pred = len(subset[(subset['two_year_recid'] == 1) & (subset['pred_recid'] == 1)]) / len(subset[subset['pred_recid'] == 1])*100

    print(f'TPR: {TPR:.0f}%, FPR: {FPR:.0f}%, TNR: {TNR:.0f}% and FNR: {FNR:.0f}% og positiv prediktiv var {pos_pred:.0f}%.')

# Fjerner kols som ikke brukes
rater_gruppe = data[['sex', 'race', 'two_year_recid', 'pred_recid', 'score_text']]
# Tar kun med hvite og sorte individer
rater_gruppe = rater_gruppe[rater_gruppe['race'].isin(['Caucasian','African-American'])]

print('For alle grupper har vi ratene:')
rate(rater_gruppe)
print('For hvite har vi ratene:')
rate(rater_gruppe[rater_gruppe['race'] == 'Caucasian'])
print('For sorte har vi ratene:')
rate(rater_gruppe[rater_gruppe['race'] == 'African-American'])
print('For kvinner har vi ratene:')
rate(rater_gruppe[rater_gruppe['sex'] == 'Female'])
print('For menn har vi ratene:')
rate(rater_gruppe[rater_gruppe['sex'] == 'Male'])


For alle grupper har vi ratene:
TPR: 65%, FPR: 33%, TNR: 67% and FNR: 35% og positiv prediktiv var 63%.
For hvite har vi ratene:
TPR: 50%, FPR: 22%, TNR: 78% and FNR: 50% og positiv prediktiv var 59%.
For sorte har vi ratene:
TPR: 72%, FPR: 42%, TNR: 58% and FNR: 28% og positiv prediktiv var 65%.
For kvinner har vi ratene:
TPR: 63%, FPR: 34%, TNR: 66% and FNR: 37% og positiv prediktiv var 52%.
For menn har vi ratene:
TPR: 65%, FPR: 33%, TNR: 67% and FNR: 35% og positiv prediktiv var 66%.
